In [50]:
%use lets-plot
%use dataframe

data class Point(val x: Long, val y: Long)
var input = java.io.File("input.txt").readText().trim()
    .split("\n").map{ it.trim().split(",") }
    .map{ Point(it[0].toLong(), it[1].toLong()) }

data class Rectangle(val topLeft: Point, val bottomRight: Point)
data class Line(val start: Point, val end: Point)

fun area(r: Rectangle): Long {
    return kotlin.math.abs(r.bottomRight.x - r.topLeft.x + 1) *
            kotlin.math.abs(r.bottomRight.y - r.topLeft.y + 1)
}

fun toRectangle(p1: Point, p2: Point): Rectangle {
    val topLeftX = kotlin.math.min(p1.x, p2.x)
    val topLeftY = kotlin.math.min(p1.y, p2.y)
    val bottomRightX = kotlin.math.max(p1.x, p2.x)
    val bottomRightY = kotlin.math.max(p1.y, p2.y)
    return Rectangle(Point(topLeftX, topLeftY), Point(bottomRightX, bottomRightY))
}

fun isHorizontal(l: Line): Boolean {
    return l.start.y == l.end.y
}

fun isVertical(l: Line): Boolean {
    return l.start.x == l.end.x
}

fun intersect(a: Line, b: Line): Boolean {
    if(a.start == b.start || a.start == b.end || a.end == b.start || a.end == b.end) {
        return false // Lines that touch at endpoints are not considered intersecting
    }

    if(isHorizontal(a))
    {
        if(isHorizontal(b)) return false // Overlapping lines are not considered to be intersecting
        else {
            // a is horizontal, b is vertical
            return (b.start.x in kotlin.math.min(a.start.x, a.end.x) + 1..kotlin.math.max(a.start.x, a.end.x) - 1) &&
                   (a.start.y in kotlin.math.min(b.start.y, b.end.y) + 1..kotlin.math.max(b.start.y, b.end.y) - 1)
        }
    } else if(isVertical(a))
    {
        if(isVertical(b)) return false // Overlapping lines are not considered to be intersecting
        else {
            // a is vertical, b is horizontal
            return (a.start.x in kotlin.math.min(b.start.x, b.end.x) + 1 ..kotlin.math.max(b.start.x, b.end.x) - 1) &&
                   (b.start.y in kotlin.math.min(a.start.y, a.end.y) + 1 ..kotlin.math.max(a.start.y, a.end.y) - 1)
        }
    } else {
        throw IllegalArgumentException("Only horizontal and vertical lines are supported")
    }
}

fun edges(r: Rectangle) : List<Line> {
    val topLeft = r.topLeft
    val bottomRight = r.bottomRight
    val topRight = Point(bottomRight.x, topLeft.y)
    val bottomLeft = Point(topLeft.x, bottomRight.y)

    return listOf(
        Line(topLeft, topRight),
        Line(topRight, bottomRight),
        Line(bottomRight, bottomLeft),
        Line(bottomLeft, topLeft)
    )
}

fun intersect(r: Rectangle, l: Line) : Boolean {
    for (edge in edges(r)) {
        if (intersect(edge, l)) {
            return true
        }
    }
    return false
}

fun inside(r: Rectangle, l: Line) : Boolean {
    return (l.start.x > r.topLeft.x && l.start.x < r.bottomRight.x &&
            l.start.y > r.topLeft.y && l.start.y < r.bottomRight.y) &&
           (l.end.x > r.topLeft.x && l.end.x < r.bottomRight.x &&
            l.end.y > r.topLeft.y && l.end.y < r.bottomRight.y)
}


var rectangles = input.flatMap { p1 ->
    input.map { p2 -> toRectangle(p1, p2) }
}

var polygon = input.zipWithNext { p1, p2 -> Line(p1, p2) } + Line(input.last(), input.first())

var df = dataFrameOf("x" to input.map{ it.x },
                     "y" to input.map{ it.y },
    "t" to input.map { "polygon" })

var data = df.toMap()
//print(data)

var validRectangles = rectangles
    .filter { r -> !polygon.any { l -> intersect(r, l) || inside(r, l) } }
    .sortedByDescending { area(it)}

var biggestValidRectangle = validRectangles.maxBy { area(it) }
println("Biggest valid rectangle found: $biggestValidRectangle with area ${area(biggestValidRectangle)}")


var rectPolygon = edges(biggestValidRectangle).flatMap(
    { line -> listOf(line.start, line.end) }
)
var rData = dataFrameOf("x" to rectPolygon.map { it.x },
                        "y" to rectPolygon.map { it.y },
    "t" to rectPolygon.map { "rectangle" })
    .toMap()

data = data.keys.map { key ->
    Pair(key, data[key]!! + rData[key]!!)
}.toMap()

val scatterPlot =
    letsPlot(data) { x = "x"; y = "y"; color = "t" } + ggsize(1600, 1500) + geomPolygon()
scatterPlot



Biggest valid rectangle found: Rectangle(topLeft=Point(x=4615, y=50322), bottomRight=Point(x=94737, y=66437)) with area 1452422268


<path d="M1346.3445658752898 723.5319921186352 L1346.3445658752898 723.5319921186352 L1346.3445658752898 706.712839271068 L1353.4962673615735 706.712839271068 L1353.4962673615735 690.5242333110216 L1343.1630612888684 690.5242333110216 L1343.3234732848225 657.3793990930773 L1342.3877366417573 657.3793990930773 L1342.3877366417573 641.0674252637334 L1339.4067470502782 641.0674252637334 L1339.4067470502782 623.6040179876122 L1343.6710326093894 623.6040179876122 L1343.6710326093894 607.1549687479377 L1340.7836166822171 607.1549687479377 L1340.7836166822171 592.5701450887595 L1329.3141589715044 592.5701450887595 L1329.3141589715044 573.8593515786297 L1336.4123897924699 573.8593515786297 L1336.4123897924699 559.1100374270549 L1326.854508366876 559.1100374270549 L1326.854508366876 541.4410170354378 L1327.9506570058952 541.4410170354378 L1327.9506570058952 527.8842589537393 L1315.5855656511058 527.8842589537393 L1315.5855656511058 511.09252118823815 L1313.460106704715 511.09252118823815 L1313.460106704715 496.4391598238948 L1305.3325655763776 496.4391598238948 L1305.3325655763776 480.2505538638484 L1301.215324346891 480.2505538638484 L1301.215324346891 462.30738265157004 L1300.7207206926994 462.30738265157004 L1300.7207206926994 450.82046326586396 L1286.0162877302473 450.82046326586396 L1286.0162877302473 436.7839412480083 L1277.2337309517645 436.7839412480083 L1277.2337309517645 417.3740631451924 L1278.503659253067 417.3740631451924 L1278.503659253067 407.80619950411494 L1261.753973342201 407.80619950411494 L1261.753973342201 389.94527353803505 L1259.4948377325152 389.94527353803505 L1259.4948377325152 374.11306364484835 L1253.4526525515803 374.11306364484835 L1253.4526525515803 363.90094557521707 L1238.9487345840707 363.90094557521707 L1238.9487345840707 344.6007278006655 L1237.545129619473 344.6007278006655 L1237.545129619473 331.2495828344629 L1227.5193798723465 331.2495828344629 L1227.5193798723465 323.3677467404523 L1210.756326295151 323.3677467404523 L1210.756326295151 305.3971604461078 L1206.5321437350283 305.3971604461078 L1206.5321437350283 292.3475813826326 L1196.1989376623233 292.3475813826326 L1196.1989376623233 281.7516521640755 L1183.2055659900473 281.7516521640755 L1183.2055659900473 273.3626370518416 L1168.1669413693576 273.3626370518416 L1168.1669413693576 261.4370763530776 L1156.8979986535874 261.4370763530776 L1156.8979986535874 246.85225269389935 L1148.0352358771274 246.85225269389935 L1148.0352358771274 238.1616716789381 L1133.7585682372194 238.1616716789381 L1133.7585682372194 223.04225391947057 L1125.0027467913956 223.04225391947057 L1125.0027467913956 217.13430373422057 L1108.6540908704146 217.13430373422057 L1108.6540908704146 205.7981672998783 L1096.7167648381692 205.7981672998783 L1096.7167648381692 191.06256068933658 L1087.0786774145984 191.06256068933658 L1087.0786774145984 182.30344196920987 L1073.0827307676097 182.30344196920987 L1053.7931882541382 181.98816852544928 L1053.7931882541382 171.15921110933027 L1041.4013615666897 171.15921110933027 L1041.4013615666897 159.4803861491614 L1029.3303588711494 159.4803861491614 L1029.3303588711494 156.53326482705302 L1012.4068932979999 156.53326482705302 L1012.4068932979999 143.77154412527216 L1000.7101852596857 143.77154412527216 L1000.7101852596857 136.89035852667485 L985.9255462992564 136.89035852667485 L985.9255462992564 123.9367322504313 L973.827808271057 123.9367322504313 L973.827808271057 116.31533943604859 L959.1902136402524 116.31533943604859 L959.1902136402524 119.38582862745466 L940.408642447302 119.38582862745466 L940.408642447302 110.77749285869163 L926.2656514706889 110.77749285869163 L926.2656514706889 98.71485674959695 L913.0985001361295 98.71485674959695 L913.0985001361295 103.95113742422654 L894.3837672748266 103.95113742422654 L894.3837672748266 97.23444231802614 L879.4654516511024 97.23444231802614 L879.4654516511024 90.03798327566847 L864.5471360273781 90.03798327566847 L864.5471360273781 81.31998717864099 L849.7892323996078 81.31998717864099